In [1]:
# Challenge 2: Have customers narrow their travel search based on temperature and precipitation
# Import dependencies
import pandas as pd
import requests
import gmaps

# Import google api id
from config import g_key



In [21]:
# Read csv file and store into a dataframe
city_data_df = pd.read_csv('data\WeatherPy_database.csv')
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rain (inches),Snow (inches)
0,0,Antalaha,MG,-14.90,50.28,68.59,85,10,8.32,clear sky,0.0,0
1,1,Hilo,US,19.73,-155.09,82.40,58,90,8.05,overcast clouds,0.0,0
2,2,Ribeira Grande,PT,38.52,-28.70,60.93,67,59,15.59,broken clouds,0.0,0
3,3,Cherskiy,RU,68.75,161.30,55.49,40,94,14.99,overcast clouds,0.0,0
4,4,Mataura,NZ,-46.19,168.86,49.15,82,87,1.68,overcast clouds,0.0,0


In [3]:
# Configure gmaps api key
gmaps.configure(api_key=g_key)


In [4]:
# Get user input on preferred min and max temperature. Use to narrow Vacation Search
min_temp = float(input('What is the  preferred minimum temperature for your vacation?'))
max_temp = float(input('What is the  preferred maximum temperature for your vacation?'))

What is the  preferred minimum temperature for your vacation?75
What is the  preferred maximum temperature for your vacation?90


In [5]:
# Determine if the user does or does not want rain on their vacation
rain_condition = input('Do you want it to be raining?(yes/no)')

# Determine if the user does or does not want snow on their vacation
snow_condition = input('Do you want it to be snowing?(yes/no)')

# Filter the city dataframe based on user input
if rain_condition == "no" and snow_condition == "no":
    vacation_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp)
                                          &(city_data_df['Max Temp'] <= max_temp)
                                          &(city_data_df['Rain (inches)'] == 0.0)
                                          &(city_data_df['Snow (inches)'] == 0)]

elif rain_condition == "no" and snow_condition == "yes":
    vacation_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp)
                                          &(city_data_df['Max Temp'] <= max_temp)
                                          &(city_data_df['Rain (inches)'] == 0.0)
                                          &(city_data_df['Snow (inches)'] >= 0)]
    
if rain_condition == "yes" and snow_condition == "no":
    vacation_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp)
                                          &(city_data_df['Max Temp'] <= max_temp)
                                          &(city_data_df['Rain (inches)'] >= 0.0)
                                          &(city_data_df['Snow (inches)'] == 0)]
    
else:
    vacation_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp)
                                          &(city_data_df['Max Temp'] <= max_temp)
                                          &(city_data_df['Rain (inches)'] >= 0.0)
                                          &(city_data_df['Snow (inches)'] >= 0)]


Do you want it to be raining?(yes/no)no
Do you want it to be snowing?(yes/no)no


In [22]:
# Create a copy of selected data in vacation_cities_df.  Add empty column 'Hotel Name'
hotels_df = vacation_cities_df[['City', 'Country', 'Max Temp', 'Description','Lat', 'Lng']].copy()
hotels_df['Hotel Name'] = " "
hotels_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Hilo,US,82.4,overcast clouds,19.73,-155.09,
8,Kapaa,US,82.4,clear sky,22.08,-159.32,
13,Thompson,CA,80.6,few clouds,55.74,-97.86,
16,Urumqi,CN,77.0,overcast clouds,43.80,87.60,
18,Victoria,HK,90.0,overcast clouds,22.29,114.16,


In [23]:
# Set parameters for gmaps search
params = {
    'radius':5000,
    'type':'lodging',
    'key':g_key
}

# Loop through each row
for index, row in hotels_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
        
    # Set the location each time through the loop
    params['location'] = f'{lat},{lng}'
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    hotels = requests.get(base_url,params=params).json()
    
    try:
        hotels_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except IndexError:
        print('Hotel not found, skipping...')
        
    

Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...
Hotel not found, skipping...


In [24]:
hotels_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Hilo,US,82.4,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
8,Kapaa,US,82.4,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
13,Thompson,CA,80.6,few clouds,55.74,-97.86,Burntwood Hotel
16,Urumqi,CN,77.0,overcast clouds,43.80,87.60,Hua Ling Grand Hotel Urumqi
18,Victoria,HK,90.0,overcast clouds,22.29,114.16,Mini Hotel Central


In [25]:
output_file_path = "data\WeatherPy_vacation.csv"
hotels_df.to_csv(output_file_path, index_label='City ID')
Vacations_df = pd.read_csv(output_file_path)
Vacations_df.head()

,City ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,1,Hilo,US,82.4,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
1,8,Kapaa,US,82.4,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
2,13,Thompson,CA,80.6,few clouds,55.74,-97.86,Burntwood Hotel
3,16,Urumqi,CN,77.0,overcast clouds,43.80,87.60,Hua Ling Grand Hotel Urumqi
4,18,Victoria,HK,90.0,overcast clouds,22.29,114.16,Mini Hotel Central


In [26]:
# Create infobox template and place hotel markers on heat layer map
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp}°F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in Vacations_df.iterrows()]
locations = Vacations_df[['Lat','Lng']]

In [27]:
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))